# Neural Networks From Scratch
## Understand how neural networks really work

Ever wondered how neural networks really work under the hood?

In this article, we'll build one from scratch using just basic matrix operations
and calculus. By the end, you should understand how neural networks work, and be
able to build one yourself from scratch.

The article is written using Jupyter Lab, and you can download the corresponding
code at
https://github.com/alan-cooney/blog/blob/main/posts/mnist-from-scratch/mnist-from-scratch.ipynb
. It assumes you know roughly what a Neural Network is (watch [this video](https://www.youtube.com/watch?v=aircAruvnKk&t=57s) first
if not), and have a basic knowledge of linear algebra and calculus.

## The challenge

The challenge we'll use is hand-written digit recognition from the famous
[MNIST](http://yann.lecun.com/exdb/mnist/) dataset. That is  to say, the model
will take images of hand-written numbers, and output the number as a single integer.

![MNIST Images are used for this example](images/Wikipedia-MNIST.png "Source:
Wikipedia")

<small>Image source: Wikipedia</small>

## Setup

We'll start by using the `tensorflow_datasets` library, as a convenient way to download the [MNIST
dataset](http://yann.lecun.com/exdb/mnist/). This will give us two datasets -
one for training the model, and another that should just be used at test time to
verify it works on never-seen numbers. We'll then simply convert them to numpy
iterators, as we won't be using TensorFlow for this article.

In [2]:
import typing
from os import getcwd, path
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from matplotlib import pyplot

# Add types for the dataset download
ds_train: tf.data.Dataset
ds_test: tf.data.Dataset

# Download
(tf_train, tf_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'], # Split into training and testing data
    data_dir=path.join(getcwd(), ".data"), # Store in .data/
    as_supervised=True, # Get tuples (features, label)
    with_info=True, # Include extra info about the dataset
)

# Convert to Numpy iterators
train_data: typing.Iterator[typing.Tuple[np.ndarray, int]] = tf_train.as_numpy_iterator()
test_data: typing.Iterator[typing.Tuple[np.ndarray, int]] = tf_test.as_numpy_iterator()

# Print the shape of an image
train_data.next()[0].shape

(28, 28, 1)

## Our Approach

Our approach will be to create a simple one-layer neural network (NN). This
means:

 1. Each image will be converted from 28x28 grayscale pixels, to a single column vector
   (784x1) of numbers. Each number in this vector will range from 0 (black) to
   255 (white). The corresponding label vector ($\bf{y}$) is the one-hot-encoded integer
   (e.g. $1$ is $[0,1,0,...,0]$).

  $$\displaystyle \bf{x} = \text{image} = 
  \begin{bmatrix} x_1 \\ ... \\ x_{784} \end{bmatrix}$$

   $$\displaystyle \bf{y} = \text{one-hot encoded label} = 
  \begin{bmatrix} y_0 \\ ... \\ y_{9} \end{bmatrix}$$

  $$\displaystyle y_i \in \{0,1\}, \sum_{i=0}^{9}y_i = 1$$

 2. We'll then initialise a matrix of random weights, of size 10 (number of
    digits 0-9) x 784 (number of pixels).

  $$\displaystyle \bf{w} = \text{weights} = 
  \begin{bmatrix} w_{0,1} & ... & w_{0,784} \\ ... & ...  & ... \\ w_{9,1} & ...& w_{9,784} \end{bmatrix}$$

 3. We'll create a simple model, that multiplies these weights by each image, to
   produce a 10x1 set of 'activations' (outputs). There is one activation per
   potential prediction (integers from 0-9).
   
$$\displaystyle \bf{w} * \bf{x} = \bf{a}  = 
 \begin{bmatrix} a_0 \\ .. \\ a_{9} \end{bmatrix}$$

 4. To convert these 10 activations into a prediction, we'll use the softmax
    function to help pick out the biggest activation (a soft maximum). The
    softmax (`S(a)`) function maps one vector to another (of the same shape), where all
    the new values sum to 1. The resulting values will be the model's prediction
    ($p$) of the probabilities that the image represents that specific integer.
 
  $$\displaystyle S(\bf{a}) :
  \begin{bmatrix} a_0 \\ ... \\ a_{9} \end{bmatrix}
  \rightarrow
  \begin{bmatrix} S_0 \\ ... \\ S_{9} \end{bmatrix}
  $$

  $$\displaystyle S_i = \frac{e^{a_i}}{\sum_{k=1}^{10}{e^{a_k}}}$$

  $$\displaystyle S_i = \text{Probability that the image is the
  integer i}$$

  $$\displaystyle \bf{p} = \text{predictions} = S(\bf{a})$$

 5. Next we'll calculate the loss (how inaccurate our predictions were), using
    cross-entropy loss ($\text{xent}(\bf{y},\bf{p})$). We'll use this loss function as it
    informally gives us the distance between two probability distributions.
    These are the correct outputs ($y$), and the predictions ($p$). This
    completes what's known as the *forward pass*.

    $$\displaystyle \text{xent}(\bf{y},\bf{p}) = -\sum_{k=0}^{9}{y_k * \ln{(p_k)}}$$

 6. Finally, to learn, we'll calculate the gradients of the loss with
    respect to the weights (derived later in this article), and then use
    gradient descent (small steps) to update the weights & reduce the loss. This
    is what's known as the *backwards pass*.
    
 7. Loop. We'll keep doing this, looping through each image in the dataset many
    times. As the weights start leading to more and more accurate predictions,
    the model can be said to be 'learning'.

### Forwards pass

#### Softmax

To begin with, we'll create the Softmax function. 

  $$\displaystyle S_i = \frac{e^{a_i}}{\sum_{k=0}^{9}{e^{a_k}}}$$

A slight complication of this is numerical stability - in that $S_i$ values can
get close to infinity which can't be represented by a `float64` number on a
computer (the maximum is $10^{308}$). To solve this, we use a well known trick
whereby we multiply the numerator and denominator by a constant, typically 
$e^{-max(x)}$ which has the effect of shifting the softmax inputs closer to 0
(without changing the output).

  $$\displaystyle S_i = 
  \frac{e^{a_i}}{\sum_{k=0}^{9}{e^{a_k}}}
  =
  \frac{e^{a_i}*e^{-max(\bf{a})}}{\sum_{k=0}^{9}{e^{a_k}*e^{-max(\bf{a})}}}
  =
  \frac{e^{a_i-max(\bf{a})}}{\sum_{k=0}^{9}{e^{a_k-max(\bf{a})}}}$$

In [3]:

def softmax(activations: np.ndarray) -> np.ndarray:
    """Softmax with numerical stability

    Args:
        activations (np.ndarray): The activation values (a)

    Returns:
        np.ndarray: Softmax vector (S)
    """
    shift_activations = activations - np.max(activations)
    numerator = np.exp(shift_activations)
    denominator = np.exp(shift_activations).sum()
    return numerator/denominator

# Example
softmax([10, 10, 1000])

array([0., 0., 1.])

### Cross entropy loss

Next we'll create the loss function.

$$\displaystyle \text{xent}(\bf{y},\bf{p}) = -\sum_{k=0}^{9}{y_k\ln(p_k)}$$

The reason cross-entropy loss is used so widely for classification problems, is
that it is very simple. This is because $y_k = 0$ where $k \neq \text{label}$, and 1
otherwise, so:

$$\displaystyle \text{xent}(\bf{y},\bf{p}) = -\ln(p_y)$$

In [4]:
from math import log

def cross_entropy_loss(label: int, predictions: np.ndarray) -> float:
    """Cross-entropy loss

    Args:
        label (int): Label (integer value of the image)
        predictions (np.ndarray): Prediction 10x1 matrix

    Returns:
        float: Cross-entropy loss
    """
    prediction_probability_label = predictions[label]
    return -log(prediction_probability_label)

#### Forward function

With that done, we can create the full forward function:

In [9]:
def forward(weights: np.ndarray, image: np.ndarray, label: int) -> float:
    """Forward pass

    Args:
        weights (np.ndarray): Weights
        image (np.ndarray): Image
        label (int): Image label (integer from 0-9)

    Returns:
        float: Loss
    """
    # First let's flatten the image (to be a vertical 784x1 matrix)
    # We'll also normalize to be floats from 0-1 rather than ints
    flattened_image = image.reshape(28*28,1) / 255.

    # Let's also one-hot-encode the label
    label = np.zeros(10).reshape((-1,1))
    label.put(label, 1)

    # Calculate the activations
    activations = np.matmul(weights, flattened_image)

    # Use softmax to calculate the predictions
    predictions = softmax(activations)

    # Calculate the loss
    loss = cross_entropy_loss(label, predictions)

    return loss

# weights = np.random.rand(10, 28*28) / 78*78
# data = train_data.next()
# res = forward(weights, data[0], data[1])
# res

### Backwards Pass

#### Chain Rule Approach

Out approach here will be to use the chain rule, to calculate the derivative of
the loss with respect to each weight:

$$\displaystyle
    \frac{\partial{\text{loss}}}{\partial{\text{weight}_{i,j}}} =
    \frac{\partial{\text{loss}}}{\partial{\text{predictions}}} *
    \frac{\partial{\text{predictions}}}{\partial{\text{activations}}} * 
    \frac{\partial{\text{activations}}}{\partial{\text{weight}_{i,j}}}$$

$$\displaystyle
    =
    \frac{\partial{\bf{l}}}{\partial{\bf{p}}} *
    \frac{\partial{\bf{p}}}{\partial{\bf{a}}} * 
    \frac{\partial{\bf{a}}}{\partial{w_{i,j}}}$$

One convenience however is that loss only depends on $p_y$ (the predicted
probability that $x$ is integer $y$), because it is defined as follows:

$$\displaystyle \bf{l} = \text{xent}(\bf{y},\bf{p}) = -\ln(p_y)$$

This means that we can simplify our derivative calculation to:

$$\displaystyle
    \frac{\partial{\bf{l}}}{\partial{p_y}} *
    \frac{\partial{p_y}}{\partial{\bf{a}}} * 
    \frac{\partial{\bf{a}}}{\partial{w_{i,j}}}$$

##### Loss w.r.t. predictions

Remembering that cross-entropy loss is calculated as follows:

$$\displaystyle \bf{l} = \text{xent}(\bf{y},\bf{p}) = -\ln(p_y)$$

The partial derivative is simply:

$$\displaystyle
\frac{\partial{\bf{l}}}{\partial{p_y}} = -\frac{1}{p_y}$$



##### Predictions w.r.t. activations

The softmax for $p_y$ was defined as follows, noting how we simplify the sum
term to be represented as just the $\sum$ symbol:

$$\displaystyle p_y = S(a_y) = \frac{e^{a_y}}{\sum_{k=0}^{9}{e^{a_k}}}
= \frac{e^{a_y}}{\sum}$$

Using the quotient rule, for a specific element $a_j$ of $\bf{a}$:

$$\displaystyle
    \frac{dy}{dx} = \frac{v*\frac{du}{dx}-u*\frac{dv}{dx}}{v^2}
$$

$$\displaystyle
    \frac{\partial{p_y}}{\partial{a_i}} =
    \frac{{\sum}*{\frac{\partial}{\partial{a_i}}e^{a_y}}-{e^{a_y}}{e^{a_i}}}{{\sum}^2}
$$

We can simplify this further, by noting that
$\frac{\partial}{\partial{a_i}}e^{a_y}$ is 1 if $y = i$ and $0$ otherwise:

$$\displaystyle
    \text{If } y = i \text{ :}
$$

$$\displaystyle
    \frac{\partial{p_y}}{\partial{a_i}} =
    \frac{{\sum}*{e^{a_y}}-{e^{a_y}}{e^{a_i}}}{{\sum}^2}
    = p_y(1 - p_i)
$$

$$\displaystyle
    \text{Else if } y \neq i \text{ :}
$$

$$\displaystyle
    \frac{\partial{p_y}}{\partial{a_i}} =
    \frac{{\sum}*{0}-{e^{a_y}}{e^{a_i}}}{{\sum}^2}
    = -p_y * p_i
$$

Given this, we can therefore calculate the gradient of $p_y$ with respect to all
values of $a$:

$$\displaystyle
\frac{\partial{\bf{l}}}{\partial{\bf{a}}} = -\frac{1}{p_y}$$

##### Activations w.r.t. weights

The activations were calculated as follows:

$$\displaystyle \bf{a}  = \bf{w} * \bf{x}$$

Given that $w_{i,j}$ is multiplied by $x_i$ (using the rules of matrix
multiplication), we get the partial derivative as follows:

$$\displaystyle
    \frac{\partial{\bf{a}}}{\partial{w_{i,j}}}= x_i$$

##### Combining with the chain rule

